In [3]:
from dotenv import load_dotenv
from os import getenv

load_dotenv()
ITS_ID = getenv("ITS_ID")
ITS_PASSWORD = getenv("ITS_PASSWORD")
PANDORA_URL = getenv("PANDORA_URL")

assert all([ITS_ID, ITS_PASSWORD, PANDORA_URL]), "Environment variables not set"

import imaplib
import email
import ssl
import logging
import time

# 设置基本的日志配置
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.header import decode_header
from email.utils import parseaddr

# Configuration settings
IMAP_SERVER = "imap.pku.edu.cn"
SMTP_SERVER = "smtp.pku.edu.cn"
IMAP_PORT = 993  # SSL port
SMTP_PORT = 25  # No SSL

EMAIL = ITS_ID + "@pku.edu.cn"
PASSWORD = ITS_PASSWORD

In [5]:
def create_ssl_context():
    """创建一个配置为 TLSv1.2 和 AES128-SHA 的 SSL 上下文"""
    context = ssl.create_default_context()
    context.options &= ~ssl.OP_NO_TLSv1_2  # 确保启用 TLSv1.2
    context.set_ciphers("AES128-SHA")  # 尝试指定 AES128-SHA 加密套件
    return context


def check_new_email(imap_server, imap_port, user_email, password, recent_min=60):
    """Check if there is a new email in the inbox."""
    logger.info("Checking for new emails...")
    new_emails = []
    with imaplib.IMAP4_SSL(
        imap_server, imap_port, ssl_context=create_ssl_context()
    ) as imap:
        # Log in to the server
        imap.login(user_email, password)

        # Select the mailbox
        imap.select("inbox")

        # Search all UNSEEN emails
        status, data = imap.search(None, "(UNSEEN)")
        if status != "OK":
            logger.error("No emails found!")
            return []

        mail_ids = data[0].split()
        mail_ids.reverse()  # Reverse the list to start with the latest email
        for mail_id in mail_ids:
            # Get the email
            status, data = imap.fetch(mail_id, "(RFC822)")
            if status != "OK":
                logger.error("Failed to fetch email")
                continue

            # Parse the email content
            msg = email.message_from_bytes(data[0][1])
            date_tuple = email.utils.parsedate_tz(msg["Date"])
            if date_tuple:
                local_date = email.utils.mktime_tz(date_tuple)
                if (time.time() - local_date) / 60 <= recent_min:
                    new_emails.append(msg)
                else:
                    logger.info(f"Break on {msg['Date']}")

        if new_emails:
            logger.info(f"{len(new_emails)} new email(s) found!")
        else:
            logger.info("No new emails found.")
        return new_emails


def forward_email(user_email, password, to_email, emails):
    """Forward an email to another email address."""
    for email in emails:
        original_from = parseaddr(email["From"])[1]
        original_to = parseaddr(email["To"])[1]
        # Decode the subject
        decoded_header = decode_header(email["Subject"])
        original_subject = (
            decoded_header[0][0].decode(decoded_header[0][1])
            if decoded_header[0][1]
            else decoded_header[0][0]
        )

        if (
            not original_to.endswith("@xiaotian.dev")
            or original_from != "noreply@tm.openai.com"
            or original_subject != "OpenAI - Verify your email"
        ):
            logger.info(
                f"[Ignore] {original_from} -> {original_to}: {original_subject}"
            )
            continue

        logger.info(
            f"[Forwarding] {original_from} -> {original_to}: {original_subject}"
        )

        msg = MIMEMultipart()
        msg["From"] = email["From"]
        msg["To"] = email["To"]
        msg["Subject"] = email["Subject"]

        real_to = original_to.replace("@xiaotian.dev", "@pku.edu.cn")

        if email.is_multipart():
            for part in email.get_payload():
                content_type = part.get_content_type()
                if content_type == "text/html":
                    msg.attach(MIMEText(part.get_payload(), "html"))
        else:
            msg.attach(MIMEText(email.get_payload(), "plain"))

        try:
            server = smtplib.SMTP("smtp.pku.edu.cn", 25)
            server.login(user_email, password)
            text = msg.as_string()
            server.sendmail(user_email, real_to, text)
            server.quit()
            logger.info("Email forwarded.")
        except smtplib.SMTPRecipientsRefused as e:
            logger.error(f"Failed to forward email to {real_to}: {e}")
        except Exception as e:
            logger.error(f"Failed to forward email: {e}")


# Main loop
while True:
    new_emails = check_new_email(IMAP_SERVER, IMAP_PORT, EMAIL, PASSWORD)
    if new_emails:
        forward_email(EMAIL, PASSWORD, "tianyp@pku.edu.cn", new_emails)
    logger.debug("Waiting for the next check...")
    time.sleep(15)  # Check every minute

INFO:__main__:Checking for new emails...
INFO:__main__:No new emails found.


KeyboardInterrupt: 